In [26]:
from __future__ import division
from math import ceil
from collections import namedtuple as struct
import swisseph as swe

In [2]:
Date = struct('Date', ['year', 'month', 'day'])
Place = struct('Place', ['latitude', 'longitude', 'timezone'])

In [3]:
sidereal_year = 365.256360417

In [4]:
_rise_flags = swe.BIT_DISC_CENTER + swe.BIT_NO_REFRACTION

In [5]:
revati_359_50 = lambda: swe.set_sid_mode(swe.SIDM_USER, 1926892.343164331, 0)
galc_cent_mid_mula = lambda: swe.set_sid_mode(swe.SIDM_USER, 1922011.128853056, 0)

set_ayanamsa_mode = lambda: swe.set_sid_mode(swe.SIDM_LAHIRI)
reset_ayanamsa_mode = lambda: swe.set_sid_mode(swe.SIDM_FAGAN_BRADLEY)

In [6]:
gregorian_to_jd = lambda date: swe.julday(date.year, date.month, date.day, 0.0)
jd_to_gregorian = lambda jd: swe.revjul(jd, swe.GREG_CAL)

In [7]:
def to_dms_prec(deg):
    d = int(deg)
    mins = (deg - d) * 60
    m = int(mins)
    s = round((mins - m) * 60, 6)
    return [d, m, s]

In [8]:
def lunar_phase(jd):
    solar_long = solar_longitude(jd)
    lunar_long = lunar_longitude(jd)
    moon_phase = (lunar_long - solar_long) % 360
    return moon_phase

In [9]:
def nakshatra_pada(longitude):
    one_star = (360 / 27) 
    one_pada = (360 / 108)
    quotient = int(longitude / one_star)
    reminder = (longitude - quotient * one_star)
    pada = int(reminder / one_pada)
    return [1 + quotient, 1 + pada]

In [10]:
def sidereal_longitude(jd, planet):
    set_ayanamsa_mode()
    longi = swe.calc_ut(jd, planet, swe.FLG_SWIEPH | swe.FLG_SIDEREAL)
    reset_ayanamsa_mode()
    return longi[0][0] % 360 # degrees

solar_longitude = lambda jd: sidereal_longitude(jd, swe.SUN)
lunar_longitude = lambda jd: sidereal_longitude(jd, swe.MOON)

In [11]:
def sunset(jd, place):
    lat, lon, tz = place
    result = swe.rise_trans(jd - tz/24, swe.SUN, lon, lat, rsmi = _rise_flags + swe.CALC_SET)
    setting = result[1][0]
    return [setting + tz/24., to_dms((setting - jd) * 24 + tz)]

In [12]:
def moonrise(jd, place):
    lat, lon, tz = place
    result = swe.rise_trans(jd - tz/24, swe.MOON, lon, lat, rsmi = _rise_flags + swe.CALC_RISE)
    rise = result[1][0]
    return to_dms((rise - jd) * 24 + tz)

In [13]:
def moonset(jd, place):
    lat, lon, tz = place
    result = swe.rise_trans(jd - tz/24, swe.MOON, lon, lat, rsmi = _rise_flags + swe.CALC_SET)
    setting = result[1][0]
    return to_dms((setting - jd) * 24 + tz)

In [14]:
def to_dms(deg):
    d, m, s = to_dms_prec(deg)
    return [d, m, int(s)]

In [15]:
def unwrap_angles(angles):
    result = angles
    for i in range(1, len(angles)):
        if result[i] < result[i-1]: result[i] += 360
    assert(result == sorted(result))
    return result

norm180 = lambda angle: (angle - 360) if angle >= 180 else angle;

norm360 = lambda angle: angle % 360

In [16]:
def inverse_lagrange(x, y, ya):
    assert(len(x) == len(y))
    total = 0
    for i in range(len(x)):
        numer = 1
        denom = 1
        for j in range(len(x)):
            if j != i:
                numer *= (ya - y[j])
                denom *= (y[i] - y[j])
        total += numer * x[i] / denom

    return total

In [17]:
def tithi(jd, place):
    tz = place.timezone
    rise = sunrise(jd, place)[0] - tz / 24
    moon_phase = lunar_phase(rise)
    today = ceil(moon_phase / 12)
    degrees_left = today * 12 - moon_phase
    offsets = [0.25, 0.5, 0.75, 1.0]
    lunar_long_diff = [ (lunar_longitude(rise + t) - lunar_longitude(rise)) % 360 for t in offsets ]
    solar_long_diff = [ (solar_longitude(rise + t) - solar_longitude(rise)) % 360 for t in offsets ]
    relative_motion = [ moon - sun for (moon, sun) in zip(lunar_long_diff, solar_long_diff) ]
    y = relative_motion
    x = offsets
    approx_end = inverse_lagrange(x, y, degrees_left)
    ends = (rise + approx_end -jd) * 24 + tz
    answer = [int(today), to_dms(ends)]
    moon_phase_tmrw = lunar_phase(rise + 1)
    tomorrow = ceil(moon_phase_tmrw / 12)
    isSkipped = (tomorrow - today) % 30 > 1
    if isSkipped:
        leap_tithi = today + 1
        degrees_left = leap_tithi * 12 - moon_phase
        approx_end = inverse_lagrange(x, y, degrees_left)
        ends = (rise + approx_end -jd) * 24 + place.timezone
        leap_tithi = 1 if today == 30 else leap_tithi
        answer += [int(leap_tithi), to_dms(ends)]
    return answer

In [18]:
def nakshatra(jd, place):
    lat, lon, tz = place
    rise = sunrise(jd, place)[0] - tz / 24
    offsets = [0.0, 0.25, 0.5, 0.75, 1.0]
    longitudes = [ lunar_longitude(rise + t) for t in offsets]
    nak = ceil(longitudes[0] * 27 / 360)
    y = unwrap_angles(longitudes)
    x = offsets
    approx_end = inverse_lagrange(x, y, nak * 360 / 27)
    ends = (rise - jd + approx_end) * 24 + tz
    answer = [int(nak), to_dms(ends)]
    nak_tmrw = ceil(longitudes[-1] * 27 / 360)
    isSkipped = (nak_tmrw - nak) % 27 > 1
    if isSkipped:
        leap_nak = nak + 1
        approx_end = inverse_lagrange(offsets, longitudes, leap_nak * 360 / 27)
        ends = (rise - jd + approx_end) * 24 + tz
        leap_nak = 1 if nak == 27 else leap_nak
        answer += [int(leap_nak), to_dms(ends)]
    return answer

In [19]:
def vaara(jd):
    return int(ceil(jd + 1) % 7)

In [20]:
'''
def nakshatra(jd, place):
    lat, lon, tz = place
    rise = sunrise(jd, place)[0] - tz / 24
    offsets = [0.0, 0.25, 0.5, 0.75, 1.0]
    longitudes = [ lunar_longitude(rise + t) for t in offsets]
    nak = ceil(longitudes[0] * 27 / 360)
    y = unwrap_angles(longitudes)
    x = offsets
    approx_end = inverse_lagrange(x, y, nak * 360 / 27)
    ends = (rise - jd + approx_end) * 24 + tz
    answer = [int(nak), to_dms(ends)]
    nak_tmrw = ceil(longitudes[-1] * 27 / 360)
    isSkipped = (nak_tmrw - nak) % 27 > 1
    if isSkipped:
        leap_nak = nak + 1
        approx_end = inverse_lagrange(offsets, longitudes, leap_nak * 360 / 27)
        ends = (rise - jd + approx_end) * 24 + tz
        leap_nak = 1 if nak == 27 else leap_nak
        answer += [int(leap_nak), to_dms(ends)]

    return answer
'''

# Tests below

In [21]:
nagpur = Place(21.1458, 79.0882, +5.5) #
date1 = gregorian_to_jd(Date(2023, 1, 5)) #yyyy, mm, dd

In [22]:
def tithi_tests():
    #print(sys._getframe().f_code.co_name)
    print(tithi(date1, nagpur))
    return

In [23]:
def sunrise(jd, place):
    lat, lon, tz = place
    #result = swe.rise_trans(jd - tz/24, swe.SUN, lon, lat, rsmi = _rise_flags + swe.CALC_RISE)
    geop = (lon,lat,0)
    result = swe.rise_trans(jd - tz/24, swe.SUN,_rise_flags + swe.CALC_RISE,geop)
    rise = result[1][0]
    return [rise + tz/24., to_dms((rise - jd) * 24 + tz)]

In [24]:
tithi_tests()

[13, [14, 7, 44]]
